# Step 1: Understanding the Data

## Datasets
- **URM (User-Rating Matrix)**: Sparse binary matrix indicating which users interacted with which items (implicit feedback).
- **ICM (Item-Content Matrix)**: Contains features describing items.
  - **150_ICM**: Contains 150 features per item, structured as tuples \( (\text{ItemID}, \text{FeatureID}, \text{Value}) \).
  - **500_ICM**: Contains 500 features per item, structured similarly to 150_ICM.
- **Assumptions**:
  - Both ICMs are sparse; missing entries are treated as zero.
  - Features represent song descriptors/tags, some of which may be normalized.

## Objective
To **select an optimal subset of features** from either **150_ICM** or **500_ICM** that enhances the performance of a **K-Nearest Neighbors (KNN) recommendation model**, comparing against two baselines:

1. **KNN trained with all features**.
2. **KNN trained with features selected via Bayesian optimization**.

---

# Step 2: Establishing Baselines

## Baseline 1: KNN with All Features
### Process
1. Construct the **full item-feature matrix**:
   - Rows: Items
   - Columns: Features
   - Missing \( (\text{ItemID}, \text{FeatureID}) \) pairs are filled with **0**.
2. Compute the **item-item similarity matrix** using **cosine similarity** with shrinkage (5) applied to the denominator:

   $$
   \text{Similarity}(i, j) = \frac{\sum_{f} v_{i,f} \cdot v_{j,f}}{\sqrt{\sum_{f} v_{i,f}^{2} \cdot \sum_{f} v_{j,f}^{2}} + 5}
   $$

   where:
   - \( v_{i,f} \) is the value of feature \( f \) for item \( i \).
   - Shrinkage term = 5 to prevent overfitting in cases with low interaction counts.

3. For each item, identify its **100 most similar items** (\( k = 100 \)).
4. Use **URM** to train the model and **predict recommendations**.
5. Evaluate using **nDCG@10** on the private test holdout.

### Output
- \( \text{nDCG@10} \) score for **150_ICM** using all 150 features.
- \( \text{nDCG@10} \) score for **500_ICM** using all 500 features.

---

## Baseline 2: KNN with Bayesian Search Feature Selection
### Process
1. **Define the search space**:
   - Each feature in the ICM can be **included (1) or excluded (0)**, forming a **binary mask**.
   - The feature space has **\( 2^{150} \) or \( 2^{500} \) possible subsets**.
2. **Optimize feature selection using Bayesian search**:
   - **Objective function**: Train KNN with the selected subset, compute item-item similarities, and evaluate \( \text{nDCG@10} \).
   - **Optimization**: Bayesian search iteratively samples feature subsets, using a **surrogate model** (e.g., **Gaussian Process**) to predict performance.
3. **Train the final KNN model** with the best-selected features.
4. **Evaluate on the test holdout using \( \text{nDCG@10} \).**

### Output
- \( \text{nDCG@10} \) score for **150_ICM** with Bayesian-selected features.
- \( \text{nDCG@10} \) score for **500_ICM** with Bayesian-selected features.

---

# Step 3: Feature Selection Strategy

Beyond baselines, we propose a **hybrid feature selection** approach using **statistical analysis** and **model evaluation**.

## Proposed Method: Hybrid Feature Selection
### Step 1: Feature Importance via Statistical Methods
1. **Compute feature variance**:
   - Features with **low variance** (mostly 0 or constant values) contribute little to distinguishing items.
2. **Calculate mutual information (MI)**:
   - Compute the **mutual information** between each feature and **URM interaction patterns** (binarized user preferences).
   - Higher MI indicates stronger predictive power for user interactions.
3. **Rank features**:
   - Combine **variance + MI** scores to generate an initial ranking.

### Step 2: Iterative Feature Subset Evaluation
1. Start with the **top \( N \) features** (e.g., \( N=50 \)).
2. Train **KNN model** and evaluate \( \text{nDCG@10} \) on a **validation split**.
3. **Iteratively refine the feature set**:
   - **Forward selection**: Add the next highest-ranked feature **if it improves \( \text{nDCG@10} \)**.
   - **Backward elimination**: Remove the least impactful feature **if its exclusion does not degrade \( \text{nDCG@10} \) significantly**.
   - Stop when adding/removing features **no longer improves performance** or when a **predefined feature limit** is reached (e.g., 100 features).




In [6]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from skopt import gp_minimize
from sklearn.metrics import ndcg_score

# Load data from CSV files
def load_urm(file_path='URM.csv'):
    urm_df = pd.read_csv(file_path)
    n_users = urm_df['UserID'].max() + 1
    n_items = urm_df['ItemID'].max() + 1
    urm = csr_matrix((np.ones(len(urm_df)), (urm_df['UserID'], urm_df['ItemID'])),
                     shape=(n_users, n_items))
    return urm, urm_df['ItemID'].unique()  # Return URM and unique ItemIDs

def load_icm(file_path, n_items, valid_item_ids):
    icm_df = pd.read_csv(file_path)
    # Filter ICM to only include ItemIDs present in URM
    icm_df = icm_df[icm_df['ItemID'].isin(valid_item_ids)]
    n_features = icm_df['FeatureID'].max() + 1
    icm = csr_matrix((icm_df['Value'], (icm_df['ItemID'], icm_df['FeatureID'])),
                     shape=(n_items, n_features))
    return icm

# Load datasets
urm, valid_item_ids = load_urm('URM.csv')
n_items = urm.shape[1]
icm_150 = load_icm('150_ICM.csv', n_items, valid_item_ids)
icm_500 = load_icm('500_ICM.csv', n_items, valid_item_ids)

# Split URM into training and validation
urm_train, urm_val = train_test_split(urm, test_size=0.2, random_state=42)

# Item-Based KNN Model
class ItemKNN:
    def __init__(self, similarity, k=100):
        self.similarity = similarity
        self.k = k

    def fit(self, urm):
        self.urm = urm
        # Precompute top-k neighbors
        self.top_k_indices = np.argpartition(self.similarity.toarray(), -self.k, axis=1)[:, -self.k:]

    def predict(self, user_ids):
        scores = self.urm[user_ids].dot(self.similarity)  # User-item scores
        preds = {}
        for i, u in enumerate(user_ids):
            user_scores = scores[i].toarray().flatten()
            top_k = np.argpartition(user_scores, -10)[-10:]  # Top-10 items
            preds[u] = top_k[np.argsort(-user_scores[top_k])]  # Sort by score
        return preds

# nDCG@10 evaluation
def evaluate_ndcg(predictions, urm_true):
    true_relevance = urm_true.toarray()
    pred_scores = np.zeros_like(true_relevance)
    for user, items in predictions.items():
        pred_scores[user, items] = np.arange(10, 0, -1)  # Assign decreasing scores
    return ndcg_score(true_relevance, pred_scores, k=10)

# Baseline 1: KNN with All Features
def compute_knn_baseline(icm, urm_train, urm_val):
    similarity = cosine_similarity(icm, dense_output=False)
    # Apply shrinkage
    diag = np.diag(similarity.toarray())
    norm = np.sqrt(diag)[:, None] * np.sqrt(diag)[None, :] + 5
    similarity = csr_matrix(similarity / norm)
    model = ItemKNN(similarity, k=100)
    model.fit(urm_train)
    predictions = model.predict(np.unique(urm_val.nonzero()[0]))
    return evaluate_ndcg(predictions, urm_val)

print("Computing Baseline 1...")
ndcg_150_all = compute_knn_baseline(icm_150, urm_train, urm_val)
ndcg_500_all = compute_knn_baseline(icm_500, urm_train, urm_val)

# Baseline 2: Bayesian Search Feature Selection
def bayesian_search(icm, urm_train, urm_val):
    def objective(feature_mask):
        subset_icm = icm[:, np.array(feature_mask, dtype=bool)]
        similarity = cosine_similarity(subset_icm, dense_output=False)
        diag = np.diag(similarity.toarray())
        norm = np.sqrt(diag)[:, None] * np.sqrt(diag)[None, :] + 5
        similarity = csr_matrix(similarity / norm)
        model = ItemKNN(similarity, k=100)
        model.fit(urm_train)
        preds = model.predict(np.unique(urm_val.nonzero()[0]))
        return -evaluate_ndcg(preds, urm_val)  # Minimize -nDCG

    result = gp_minimize(objective, [(0, 1)] * icm.shape[1], n_calls=30, random_state=42)
    best_features = np.array(result.x, dtype=bool)
    similarity = cosine_similarity(icm[:, best_features], dense_output=False)
    diag = np.diag(similarity.toarray())
    norm = np.sqrt(diag)[:, None] * np.sqrt(diag)[None, :] + 5
    similarity = csr_matrix(similarity / norm)
    model = ItemKNN(similarity, k=100)
    model.fit(urm_train)
    preds = model.predict(np.unique(urm_val.nonzero()[0]))
    return evaluate_ndcg(preds, urm_val), best_features

print("Computing Baseline 2...")
ndcg_150_bayes, best_features_150 = bayesian_search(icm_150, urm_train, urm_val)
ndcg_500_bayes, best_features_500 = bayesian_search(icm_500, urm_train, urm_val)

# Hybrid Feature Selection
def hybrid_feature_selection(icm, urm_train, urm_val):
    variances = np.var(icm.toarray(), axis=0)
    item_interactions = urm_train.sum(axis=0).A1
    mi_scores = mutual_info_classif(icm, item_interactions > 0, random_state=42)
    feature_scores = variances + mi_scores
    ranked_features = np.argsort(feature_scores)[::-1]

    current_features = ranked_features[:50].tolist()
    best_ndcg = 0
    best_subset = current_features.copy()

    for i in range(50, icm.shape[1]):
        subset_icm = icm[:, current_features]
        similarity = cosine_similarity(subset_icm, dense_output=False)
        diag = np.diag(similarity.toarray())
        norm = np.sqrt(diag)[:, None] * np.sqrt(diag)[None, :] + 5
        similarity = csr_matrix(similarity / norm)
        model = ItemKNN(similarity, k=100)
        model.fit(urm_train)
        preds = model.predict(np.unique(urm_val.nonzero()[0]))
        ndcg = evaluate_ndcg(preds, urm_val)
        if ndcg > best_ndcg:
            best_ndcg = ndcg
            best_subset = current_features.copy()
            current_features.append(ranked_features[i])
        else:
            break

    similarity = cosine_similarity(icm[:, best_subset], dense_output=False)
    diag = np.diag(similarity.toarray())
    norm = np.sqrt(diag)[:, None] * np.sqrt(diag)[None, :] + 5
    similarity = csr_matrix(similarity / norm)
    model = ItemKNN(similarity, k=100)
    model.fit(urm_train)
    preds = model.predict(np.unique(urm_val.nonzero()[0]))
    return evaluate_ndcg(preds, urm_val), best_subset

print("Computing Hybrid Feature Selection...")
ndcg_150_hybrid, best_features_150_hybrid = hybrid_feature_selection(icm_150, urm_train, urm_val)
ndcg_500_hybrid, best_features_500_hybrid = hybrid_feature_selection(icm_500, urm_train, urm_val)

# Final training on full URM for submission
# Compute similarity with shrinkage once and reuse
similarity_150 = cosine_similarity(icm_150[:, best_features_150_hybrid], dense_output=False)
diag_150 = np.diag(similarity_150.toarray())
norm_150 = np.sqrt(diag_150)[:, None] * np.sqrt(diag_150)[None, :] + 5
final_similarity_150 = csr_matrix(similarity_150 / norm_150)
final_model_150 = ItemKNN(final_similarity_150, k=100)
final_model_150.fit(urm)

similarity_500 = cosine_similarity(icm_500[:, best_features_500_hybrid], dense_output=False)
diag_500 = np.diag(similarity_500.toarray())
norm_500 = np.sqrt(diag_500)[:, None] * np.sqrt(diag_500)[None, :] + 5
final_similarity_500 = csr_matrix(similarity_500 / norm_500)
final_model_500 = ItemKNN(final_similarity_500, k=100)
final_model_500.fit(urm)

# Output results
print(f"Baseline 1 (All Features): 150_ICM: {ndcg_150_all:.4f}, 500_ICM: {ndcg_500_all:.4f}")
print(f"Baseline 2 (Bayesian): 150_ICM: {ndcg_150_bayes:.4f}, 500_ICM: {ndcg_500_bayes:.4f}")
print(f"Hybrid Selection: 150_ICM: {ndcg_150_hybrid:.4f}, 500_ICM: {ndcg_500_hybrid:.4f}")
print(f"Selected features (150_ICM): {len(best_features_150_hybrid)} features")
print(f"Selected features (500_ICM): {len(best_features_500_hybrid)} features")

Computing Baseline 1...
Computing Baseline 2...
Computing Hybrid Feature Selection...


/Users/jiangqinma/Documents/Qclef/.venv/lib/python3.11/site-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/Users/jiangqinma/Documents/Qclef/.venv/lib/python3.11/site-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/Users/jiangqinma/Documents/Qclef/.venv/lib/python3.11/site-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/Users/jiangqinma/Documents/Qclef/.venv/lib/python3.11/site-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received 

Baseline 1 (All Features): 150_ICM: 0.0043, 500_ICM: 0.0048
Baseline 2 (Bayesian): 150_ICM: 0.0054, 500_ICM: 0.0055
Hybrid Selection: 150_ICM: 0.0044, 500_ICM: 0.0054
Selected features (150_ICM): 51 features
Selected features (500_ICM): 51 features


Analyzing the Results

Baseline 1 (All Features): Using all features gives the lowest scores (0.0043 for 150_ICM, 0.0048 for 500_ICM), indicating that including all features introduces noise.

Baseline 2 (Bayesian): Achieves the highest scores (0.0054 and 0.0055), showing that optimizing feature selection improves performance.

Hybrid Selection: Performs slightly better than Baseline 1 but not as well as Baseline 2 (0.0044 for 150_ICM, 0.0054 for 500_ICM), suggesting that the heuristic approach (variance + mutual information) is decent but not optimal.